## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


In [1]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'HourlyRate', 'JobInvolvement', 'JobSatisfaction', 'WorkLifeBalance',
    'YearsAtCompany', 'YearsWithCurrManager'
]


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.info()


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [2]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_train.info(), X_test.info()

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Scale the training and testing data

X_train_scaled[:5], X_test_scaled[:5]

In [3]:
# Create a OneHotEncoder for the Department column
encoder_dept = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
dept_train_encoded = encoder_dept.fit_transform(y_train[['Department']])
dept_test_encoded = encoder_dept.transform(y_test[['Department']])
# Create two new variables by applying the encoder
# to the training and testing data
dept_train_encoded[:5], dept_test_encoded[:5]



In [4]:
# Create a OneHotEncoder for the Attrition column
encoder_attr = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
attr_train_encoded = encoder_attr.fit_transform(y_train[['Attrition']])
attr_test_encoded = encoder_attr.transform(y_test[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attr_train_encoded[:5], attr_test_encoded[:5]


## Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]
input_dim

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden_layer = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
dept_output_layer = layers.Dense(3, activation='softmax', name='Department')(dept_hidden_layer)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden_layer = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attr_output_layer = layers.Dense(2, activation='softmax', name='Attrition')(attr_hidden_layer)


In [5]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output_layer, attr_output_layer])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'Department': 'categorical_crossentropy', 'Attrition': 'categorical_crossentropy'},
    metrics={'Department': 'accuracy', 'Attrition': 'accuracy'}
)


# Summarize the model
model.summary()

In [6]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'Department': dept_train_encoded, 'Attrition': attr_train_encoded},
    validation_data=(X_test_scaled, {'Department': dept_test_encoded, 'Attrition': attr_test_encoded}),
    epochs=20,
    batch_size=32
)


In [7]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'Department': dept_test_encoded, 'Attrition': attr_test_encoded}
)
evaluation

In [8]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {evaluation[3]}")
print(f"Attrition Accuracy: {evaluation[4]}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 